# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from sklearn.externals import joblib
import re
import nltk

from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet') # for lemmatization

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [8]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Message', engine)
X = df.message
y = df.iloc[:,5:]
df.head()

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
print(X.size,y.size)
y.head()

26216 943776


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(sentence):
    '''
    tokenizes a sentence after normalizing it and returns lemmatized tokens.
    '''
    # normalizing, tokenizing, lemmatizing the sentence
    sentence = re.sub('\W',' ',sentence)
    tokens = word_tokenize(sentence.lower().strip())
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(i,'n') for i in tokens]
    tokens = [lemmatizer.lemmatize(i,'v') for i in tokens]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer = tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
print(X_train.shape,y_train.shape, X_test.shape,y_test.shape)
pipeline.fit(X_train,y_train)

(19662,) (19662, 36) (6554,) (6554, 36)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
def classification_rep(y_test, y_pred):
    for i, col in enumerate(y_test.columns): 
        print('-------:',col,':-------')
        print(classification_report(y_test.iloc[:,i], y_pred[:,i]))
        
y_pred = pipeline.predict(X_test)
classification_rep(y_test, y_pred)


-------: related :-------
             precision    recall  f1-score   support

          0       0.62      0.37      0.46      1587
          1       0.82      0.93      0.87      4932
          2       0.34      0.34      0.34        35

avg / total       0.77      0.79      0.77      6554

-------: request :-------
             precision    recall  f1-score   support

          0       0.89      0.98      0.94      5461
          1       0.83      0.41      0.55      1093

avg / total       0.88      0.89      0.87      6554

-------: offer :-------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.00      0.00      0.00        31

avg / total       0.99      1.00      0.99      6554

-------: aid_related :-------
             precision    recall  f1-score   support

          0       0.73      0.88      0.80      3913
          1       0.75      0.52      0.61      2641

avg / total       0.74      0.74   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
parameters = {
    'tfidf__norm':['l2','l1'],
    'clf__estimator__min_samples_split':[2,3],
    }

clf_grid = GridSearchCV(pipeline, parameters)
clf_grid.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'tfidf__norm': ['l2', 'l1'], 'clf__estimator__min_samples_split': [2, 3]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
y_pred = clf_grid.predict(X_test)
print(clf_grid.best_params_)

{'clf__estimator__min_samples_split': 2, 'tfidf__norm': 'l1'}


In [18]:
accuracy = (y_pred == y_test).sum()/y_test.shape[0]
print('accuracy :', accuracy)
classification_rep(y_test, y_pred)

accuracy : related                   0.792493
request                   0.884193
offer                     0.995270
aid_related               0.736955
medical_help              0.921269
medical_products          0.950870
search_and_rescue         0.975740
security                  0.980928
military                  0.969484
child_alone               1.000000
water                     0.948734
food                      0.926305
shelter                   0.927830
clothing                  0.984895
money                     0.978486
missing_people            0.989167
refugees                  0.966280
death                     0.961092
other_aid                 0.871834
infrastructure_related    0.936985
transport                 0.954837
buildings                 0.953158
electricity               0.980470
tools                     0.991913
hospitals                 0.989777
shops                     0.995270
aid_centers               0.988099
other_infrastructure      0.956057
weather_r

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [19]:
# use AdaBoost
adaboost_pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer = tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators=50, learning_rate = 1)))
])


In [21]:
adaboost_pipeline.fit(X_train, y_train)

y_pred = adaboost_pipeline.predict(X_test)
classification_rep(y_test, y_pred)

-------: related :-------
             precision    recall  f1-score   support

          0       0.63      0.26      0.36      1587
          1       0.80      0.95      0.87      4932
          2       0.19      0.14      0.16        35

avg / total       0.75      0.78      0.74      6554

-------: request :-------
             precision    recall  f1-score   support

          0       0.92      0.96      0.94      5461
          1       0.76      0.56      0.64      1093

avg / total       0.89      0.90      0.89      6554

-------: offer :-------
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.00      0.00      0.00        31

avg / total       0.99      0.99      0.99      6554

-------: aid_related :-------
             precision    recall  f1-score   support

          0       0.77      0.87      0.81      3913
          1       0.76      0.61      0.67      2641

avg / total       0.76      0.76   

In [22]:
# we will use gridsearch on AdaBoostClassifier to optimize it.
parameters = {
    'vect__stop_words': ['english',None],
    'tfidf__use_idf' :[True, False],
    'clf__estimator__n_estimators' : [60,70],
    'clf__estimator__learning_rate' :[1,0.5,2],
    }
clf_grid = GridSearchCV(adaboost_pipeline, parameters,verbose=2)

In [24]:
clf_grid.fit(X_train, y_train)
print(clf_grid.best_params_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=english 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=english - 1.4min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=english 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min remaining:    0.0s


[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=english - 1.4min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=english 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=english - 1.4min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=None 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=None - 2.1min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=None 
[CV]  clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=None - 2.1min
[CV] clf__estimator__learning_rate=1, clf__estimator__n_estimators=60, tfidf__use_idf=True, vect__stop_words=None 
[CV]  clf__estimator__learning_rate

[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 123.4min finished


{'clf__estimator__learning_rate': 0.5, 'clf__estimator__n_estimators': 70, 'tfidf__use_idf': True, 'vect__stop_words': None}


### 9. Export your model as a pickle file

In [26]:
import pickle
filename = 'final_model.pkl'
pickle.dump(clf_grid.best_estimator_, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [25]:
clf_grid.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=0.5, n_estimators=70, random_state=None),
           n_jobs=1))])